# Coding Tutorial 14: Foundation Model Tuning

```
Course: CSCI 5922 Spring 2025, University of Colorado Boulder
TA: Everley Tseng
Email: Yu-Yun.Tseng@colorado.edu
* AI assistant is used in making this tutorial
```

## Overview

Sections:
- LoRA (PEFT example)
- DPO (RLHF example)

Objectives:
- Learn how to leverage the `trl` library for PEFT and RLHF fine-tuning

In this tutorial, we will introduce two fine-tuning methods, PEFT (with LoRA) and RLHF (using DPO). The codes are adopted from this [GitHub site](https://github.com/huggingface/smol-course/tree/main). We have limited computing resource for model training on Colab, so please visit the GitHub site for more practices.

To fine-tune a foundation model, we recommend using more powerful GPUs. However, for testing this codebase, feel free to use the default free CPU setup on Colab.

In [ ]:
!pip install datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## LoRA Adapters

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch


The [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) from `trl` provides integration with LoRA adapters through the [PEFT](https://huggingface.co/docs/peft/en/index) library. Key advantages of this setup include:

1. **Memory Efficiency**:
   - Only adapter parameters are stored in GPU memory
   - Base model weights remain frozen and can be loaded in lower precision
   - Enables fine-tuning of large models on consumer GPUs

2. **Training Features**:
   - Native PEFT/LoRA integration with minimal setup
   - Support for QLoRA (Quantized LoRA) for even better memory efficiency

3. **Adapter Management**:
   - Adapter weight saving during checkpoints
   - Features to merge adapters back into base model

The setup requires just a few configuration steps:
1. Define the LoRA configuration (rank, alpha, dropout)
2. Create the SFTTrainer with PEFT config
3. Train and save the adapter weights


### Prepare Data

In [ ]:
dataset = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

### Prepare Model

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

### Arguments Steup

The `SFTTrainer`  supports a native integration with `peft`, which makes the tuning efficient. For the LoRA setup, we need to define the configuration, `LoraConfig`.


In [ ]:
from peft import LoraConfig

# TODO: Configure LoRA parameters
# r: rank dimension for LoRA update matrices (smaller = more compression)
rank_dimension = 6
# lora_alpha: scaling factor for LoRA layers (higher = stronger adaptation)
lora_alpha = 8
# lora_dropout: dropout probability for LoRA layers (helps prevent overfitting)
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # Rank dimension - typically between 4-32
    lora_alpha=lora_alpha,  # LoRA scaling factor - typically 2x rank
    lora_dropout=lora_dropout,  # Dropout probability for LoRA layers
    bias="none",  # Bias type for LoRA. the corresponding biases will be updated during training.
    target_modules="all-linear",  # Which modules to apply LoRA to
    task_type="CAUSAL_LM",  # Task type for model architecture
)

Finally, define the training hyperparameters in `SFTConfig`.

In [ ]:
# Hyperparameters based on QLoRA paper recommendations
args = SFTConfig(
    # Directory to save model checkpoints
    output_dir=finetune_name,
    # Training duration
    num_train_epochs=1,
    # Batch size settings
    per_device_train_batch_size=2,
    # Accumulate gradients for larger effective batch
    gradient_accumulation_steps=2,
    # Memory optimization
    gradient_checkpointing=True,
    # Optimizer settings (AdamW)
    optim="adamw_torch_fused",
    learning_rate=2e-4,  # from QLoRA paper
    max_grad_norm=0.3,  # Gradient clipping threshold
    # Portion of steps for warmup
    warmup_ratio=0.03,
    # Keep learning rate constant after warmup
    lr_scheduler_type="constant",
    # Logging and saving
    logging_steps=10,
    # Save checkpoint every epoch
    save_strategy="epoch",
    # Precision settings for bfloat16
    bf16=True,
    # Integration settings (push to hugging face)
    push_to_hub=False,
    # Disable external logging
    report_to="none",
)

Prepare the trainer `SFTTrainer` to run the training.

In [ ]:
# Create SFTTrainer with LoRA configuration
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,  # LoRA configuration
    tokenizer=tokenizer,
)

<ipython-input-19-ae79f3b17845>:16: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/2260 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

In [ ]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3639: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  ctx_manager = torch.cpu.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss


In [ ]:
# save model (if needed)
# trainer.save_model()

## Reinforcement Learning from Human Feedback

In [ ]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig

Reinforcement Learning from Human Feedback (RLHF) is a powerful method for aligning machine learning models, particularly large language models, with human preferences. By leveraging human feedback, RLHF aims to improve model behavior, often in ways that are difficult to achieve through traditional reward functions. Several methods have been developed to implement RLHF, each with its own strengths and weaknesses.

Below are some common methods:
1. **Proximal Policy Optimization (PPO)**: PPO is one of the most widely used reinforcement learning algorithms for fine-tuning models using human feedback. It combines policy optimization with a mechanism to prevent overly large updates, ensuring that the model’s behavior remains stable.
2. **Direct Preference Optimization (DPO)**: DPO is a more recent approach that directly optimizes models based on preference feedback from humans rather than using reward signals. It uses ranked feedback or pairwise comparisons to guide the optimization.






- **Model**: We will use the [`SmolLM2-135M-Instruct`](https://huggingface.co/HuggingFaceTB/SmolLM2-135M-Instruct) model which has already been trained through a SFT training, so it it compatible with DPO. You can also train your own base model following this [tutorial](https://colab.research.google.com/github/huggingface/smol-course/blob/main/1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).
- **Dataset**: We will use this TRL dataset [`trl-lib/ultrafeedback_binarized`](https://huggingface.co/datasets/trl-lib/ultrafeedback_binarized/viewer?views%5B%5D=train) dataset for fine-tuning.

### Prepare Data

In [ ]:
# Download data
dataset = load_dataset(path="trl-lib/ultrafeedback_binarized", split="train")

### Prepare Model

In [ ]:
# Load model
model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.float32,
).to(device)
model.config.use_cache = False

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Model name (after fine-tuning)
finetune_name = "SmolLM2-FT-DPO"

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

### Arguments Setup

In [ ]:
# Training arguments
training_args = DPOConfig(
    # Training batch size per GPU
    per_device_train_batch_size=4,
    # Number of updates steps to accumulate before performing a backward/update pass
    # Effective batch size = per_device_train_batch_size * gradient_accumulation_steps
    gradient_accumulation_steps=4,
    # Saves memory by not storing activations during forward pass
    # Instead recomputes them during backward pass
    gradient_checkpointing=True,
    # Base learning rate for training
    learning_rate=5e-5,
    # Learning rate schedule - 'cosine' gradually decreases LR following cosine curve
    lr_scheduler_type="cosine",
    # Total number of training steps
    max_steps=10, # 200
    # Disables model checkpointing during training
    save_strategy="no",
    # How often to log training metrics
    logging_steps=1,
    # Directory to save model outputs
    output_dir="smol_dpo_output",
    # Number of steps for learning rate warmup
    warmup_steps=5, # 100
    # Use bfloat16 precision for faster training
    bf16=True,
    # Disable wandb/tensorboard logging
    report_to="none",
    # Keep all columns in dataset even if not used
    remove_unused_columns=False,
    # Enable MPS (Metal Performance Shaders) for Mac devices
    use_mps_device=device == "mps",
    # Model ID for HuggingFace Hub uploads
    hub_model_id=finetune_name,
    # DPO-specific temperature parameter that controls the strength of the preference model
    # Lower values (like 0.1) make the model more conservative in following preferences
    beta=0.1,
    # Maximum length of the input prompt in tokens
    max_prompt_length=1024,
    # Maximum combined length of prompt + response in tokens
    max_length=1536,
)

In [ ]:
trainer = DPOTrainer(
    # The model to be trained
    model=model,
    # Training configuration from above
    args=training_args,
    # Dataset containing preferred/rejected response pairs
    train_dataset=dataset,
    # Tokenizer for processing inputs
    processing_class=tokenizer,
)

Extracting prompt in train dataset:   0%|          | 0/62135 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/62135 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/62135 [00:00<?, ? examples/s]

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Save the model (if needed)
# trainer.save_model(f"./{finetune_name}")

### [Optional] Link to Hugging Face Account

In both of the above exercises and other foundation model fine-tuning, you can save the model to your Hugging Face account. To do so, uncomment the following cells. You will be asked to authenticate and input the access token. For token creation, see Coding Tutorial 13 for reference. The name of the token is used below as `HF_TOKEN`.

In [ ]:
# !pip install huggingface_hub

In [ ]:
# Authenticate to Hugging Face

# from huggingface_hub import login
# login()

# Save to the huggingface hub if login (HF_TOKEN is set)

# if os.getenv("HF_TOKEN"):
#     trainer.push_to_hub()

## Review

We recommend visiting this [GitHub page](https://github.com/huggingface/smol-course/tree/main) for more discussions of PEFT and RLHF methods.

### Refereces

- `finetuning_sft_peft.ipynb`: https://colab.research.google.com/github/huggingface/smol-course/blob/main/1_instruction_tuning/notebooks/sft_finetuning_example.ipynb#scrollTo=BwXafdSU_ZAj
- `dpo_funetuning_example`: https://github.com/huggingface/smol-course/blob/main/2_preference_alignment/notebooks/dpo_finetuning_example.ipynb

For any questions and discussions regarding this tutorial, attend [TA office hours](https://docs.google.com/spreadsheets/d/1fzfTJpEF7RaUYRA_NGa3DkiazdQXVj7QNBbp6DrEZ3I/edit?usp=sharing) or create a post on [Piazza](https://piazza.com/colorado/spring2025/csci5922/home) :) See you in the next tutorial!

\- Everley